In [1]:
pip install gensim

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'c:\\Users\\aelazzouzi\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\numpy-1.26.4.dist-info\\INSTALLERrfvlyldu.tmp'


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.6 kB 165.2 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 359.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.0 MB 3.4 MB/s eta 0:00:08
   ---------------------------------------- 0.1/24.0 MB 1.3 MB/s eta 0:00:19
   ---------------------------------------- 0.1/24.0 MB 1.0 MB/s eta 0:00:24
   ---------------------------------------- 0.2/24.0 MB 1.5 MB/s eta 0:00:16
   ---------------------------------------- 0.3/24.0 MB 1.3 MB/s eta 0:00:19
    --------------------

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import string
import os
import re
import unicodedata
from gensim.models import Word2Vec


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

## Lets load the dataset

In [3]:
df = pd.read_csv(os.getcwd()+'\\incidents_reseau.csv')
df

,ID d'incident,Timestamp,Type d'incident,Description,Severité,Statut,Responsable
0,1.0,2023-09-24,Intrusion,échec mise jour logiciel,Élevée,Ouvert,Equipe Réseau
1,2.0,2023-07-12,Panne matérielle,mise jour firmware a échoué,Élevée,En cours,Equipe Sécurité
2,3.0,2023-05-26,Intrusion,carte réseau ne répond plus,Faible,En cours,Equipe Réseau
3,4.0,2023-04-16,Intrusion,échec mise jour logiciel,Moyenne,Ouvert,Equipe Support
4,5.0,2023-10-22,Intrusion,échec mise jour logiciel,Faible,Ouvert,Equipe Réseau
...,...,...,...,...,...,...,...
3635,NaN,NaN,Intrusion,base données est sous forte charge,NaN,NaN,NaN
3636,NaN,NaN,Surcharge,disque dur est tombé panne,NaN,NaN,NaN
3637,NaN,NaN,Surcharge,disque dur est tombé panne,NaN,NaN,NaN
3638,NaN,NaN,Surcharge,connexion au vpn a échoué,NaN,NaN,NaN


## Data Preprocessing

In [4]:
# Convertir les timestamps en datetime
#df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [5]:
# Traitement de texte
# Télécharger les stopwords de NLTK
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
stemmer = SnowballStemmer("french")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aelazzouzi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
def preprocess_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # supprimer les nombres
    text = re.sub(r'\d+', '', text)
    # normaliser les lettres qui ont des accents
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    # Appliquer le stemming
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    
    return text

df['Description'] = df['Description'].apply(preprocess_text)

In [18]:
df

,ID d'incident,Timestamp,Type d'incident,Description,Severité,Statut,Responsable
0,1.0,2023-09-24,Intrusion,echec mis jour logiciel,Élevée,Ouvert,Equipe Réseau
1,2.0,2023-07-12,Panne matérielle,mis jour firmwar a echou,Élevée,En cours,Equipe Sécurité
2,3.0,2023-05-26,Intrusion,cart reseau repond plus,Faible,En cours,Equipe Réseau
3,4.0,2023-04-16,Intrusion,echec mis jour logiciel,Moyenne,Ouvert,Equipe Support
4,5.0,2023-10-22,Intrusion,echec mis jour logiciel,Faible,Ouvert,Equipe Réseau
...,...,...,...,...,...,...,...
3635,NaN,NaN,Intrusion,bas donne sous fort charg,NaN,NaN,NaN
3636,NaN,NaN,Surcharge,disqu dur tomb pann,NaN,NaN,NaN
3637,NaN,NaN,Surcharge,disqu dur tomb pann,NaN,NaN,NaN
3638,NaN,NaN,Surcharge,connexion vpn a echou,NaN,NaN,NaN


In [7]:
# Vectorisation des descriptions
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Description'])

In [ ]:
sentences = [desc.split() for desc in df['Description']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
# Exemple pour récupérer les vecteurs
X_word2vec = [word2vec_model.wv[desc.split()].mean(axis=0) for desc in df['Description']]


## EDA


## Train Test Split


In [8]:
# Target
y = df["Type d'incident"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


## Model Training

In [10]:
models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}

In [11]:

models = {
    "Naive Bayes": MultinomialNB(),
    "Logistic Regression": LogisticRegression(max_iter=200),
    "SVM": SVC(),
    "Random Forest": RandomForestClassifier(n_estimators=100)
}# Entrainer et evaluer chaque modele
results = {}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    results[model_name] = {
        "accuracy": accuracy,
        "report": report
    }

In [12]:
# Results
for model_name, metrics in results.items():
    print(f"Model: {model_name}")
    print(f"Accuracy: {metrics['accuracy']}")
    print("Classification Report:")
    print(metrics['report'])
    print("="*60)

Model: Naive Bayes
Accuracy: 0.1709211986681465
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       0.21      0.34      0.26       177
  Mise à jour échouée       0.12      0.12      0.12       179
     Panne matérielle       0.22      0.07      0.11       183
Problème de connexion       0.17      0.10      0.13       178
            Surcharge       0.16      0.22      0.18       184

             accuracy                           0.17       901
            macro avg       0.18      0.17      0.16       901
         weighted avg       0.18      0.17      0.16       901

Model: Logistic Regression
Accuracy: 0.1753607103218646
Classification Report:
                       precision    recall  f1-score   support

            Intrusion       0.20      0.21      0.20       177
  Mise à jour échouée       0.12      0.12      0.12       179
     Panne matérielle       0.23      0.16      0.19       183
Problème de connexion      

## Pickling The Model file For Deployment

In [13]:
import pickle

In [15]:
models_dir = os.path.join(os.getcwd(), "models")
vectorizer_dir = os.path.join(os.getcwd(), "vectorizer")
for model_name, model in models.items():
    path = os.path.join(models_dir, f"{model_name}_model.pkl")
    with open(path , 'wb') as file:
        pickle.dump(model, file)

# Sauvegarder le vectorizer
path_vectorizer = os.path.join(vectorizer_dir, f"vectorizer.pkl")
with open(path_vectorizer, 'wb') as file:
    pickle.dump(vectorizer, file)

print("Models and vectorizer have been saved.")

Models and vectorizer have been saved.
